**Preparing Environment**

In [ ]:
import os
import time
from PIL import Image
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
os.environ["TFHUB_DOWNLOAD_PROGRESS"] = "True"


In [ ]:
# Declaring Constants
IMAGE_PATH = "/home/gregrevill/code/GregRevFr/photopocalypse/raw_data/input_images/champboy/boy_champagne_resized_128x128.jpg"
SAVED_MODEL_PATH = "https://tfhub.dev/captain-pool/esrgan-tf2/1"


**Defining Helper Functions**

In [ ]:
def preprocess_image(image_path):
  """ Loads image from path and preprocesses to make it model ready
      Args:
        image_path: Path to the image file
  """
  hr_image = tf.image.decode_image(tf.io.read_file(image_path))
  # If PNG, remove the alpha channel. The model only supports
  # images with 3 color channels.
  if hr_image.shape[-1] == 4:
    hr_image = hr_image[...,:-1]
  hr_size = (tf.convert_to_tensor(hr_image.shape[:-1]) // 4) * 4
  hr_image = tf.image.crop_to_bounding_box(hr_image, 0, 0, hr_size[0], hr_size[1])
  hr_image = tf.cast(hr_image, tf.float32)
  return tf.expand_dims(hr_image, 0)


In [ ]:
def save_image(image, filename):
  """
    Saves unscaled Tensor Images.
    Args:
      image: 3D image tensor. [height, width, channels]
      filename: Name of the file to save.
  """
  if not isinstance(image, Image.Image):
    image = tf.clip_by_value(image, 0, 255)
    image = Image.fromarray(tf.cast(image, tf.uint8).numpy())
  image.save("%s.jpg" % filename)
  print("Saved as %s.jpg" % filename)


In [ ]:
%matplotlib inline
def plot_image(image, title=""):
  """
    Plots images from image tensors.
    Args:
      image: 3D image tensor. [height, width, channels].
      title: Title to display in the plot.
  """
  image = np.asarray(image)
  image = tf.clip_by_value(image, 0, 255)
  image = Image.fromarray(tf.cast(image, tf.uint8).numpy())
  plt.imshow(image)
  plt.axis("off")
  plt.title(title)


#### Performing Super Resolution of images loaded from path

In [ ]:
hr_image = preprocess_image(IMAGE_PATH)


In [ ]:
model = hub.load(SAVED_MODEL_PATH)


In [ ]:
# show original image
plot_image(tf.squeeze(hr_image), title="Original")


In [ ]:
# generate super resolution image and show it
start = time.time()
fake_image = model(hr_image)
fake_image = tf.squeeze(fake_image)
run_time = time.time() - start
print("Time Taken: %f" % run_time)
plot_image(tf.squeeze(fake_image), title="Super Resolution")


In [ ]:
'''
# save super resolution image with file name including runtime

#path to save super resolution image
path = '/home/gregrevill/code/GregRevFr/photopocalypse/raw_data/output_images'

#file name to save super resolution image with : original file name + runtime
filename = os.path.basename(IMAGE_PATH).split(".")[0]
filename = filename + "_" + str(round(run_time,2)) + "secs.jg"
filepath = os.path.join(path, filename)

save_image(tf.squeeze(fake_image), filename=filepath)
'''


In [ ]:
'''
import os
import tensorflow as tf

# Function definitions for preprocess_image, model, save_image, etc. should be here

# run the code on a folder of images
folder_path = "/home/gregrevill/code/GregRevFr/photopocalypse/raw_data/input_images/champboy"

# get all file names in the folder
filenames = os.listdir(folder_path)

# get full paths to all files
filepaths = [os.path.join(folder_path, f) for f in filenames]

# sort the filepaths by file size (smallest first)
filepaths.sort(key=lambda x: os.path.getsize(x))

# save super resolution images in a folder named "super_resolution"
output_folder_path = "/home/gregrevill/code/GregRevFr/photopocalypse/raw_data/input_images/champboy_superresolution"

# process and save images
for filepath in filepaths:
    start = time.time()
    hr_image = preprocess_image(filepath)
    fake_image = model(hr_image)
    fake_image = tf.squeeze(fake_image)
    filename = os.path.basename(filepath).split(".")[0]
    run_time = time.time() - start
    filename = filename + "_" + str(round(run_time, 2)) + "secs"
    output_filepath = os.path.join(output_folder_path, filename)
    save_image(fake_image, filename=output_filepath)


In [ ]:
#install keras
from tensorflow import keras
from tensorflow.keras import layers

# Create a model instance
SAVED_MODEL_PATH = "https://tfhub.dev/captain-pool/esrgan-tf2/1"
model = hub.load(SAVED_MODEL_PATH)

#save model as .h5 keras model
model.save("esrgan.h5")
